In [11]:
from cvxpy import *
import numpy as np

# Problem setup
N = 8  # Number of beam positions
P_max = 250  # Maximum power (W)
noise_power = 1e-10  # Noise power (W)
h = np.random.rand(N, N)  # Channel gain matrix (NxN)
B = 200e6  # Bandwidth (Hz)
T_b = 1e-3  # Time slot duration (s)
alpha = np.random.rand(N)  # Slack parameter alpha

# Variables

# Variables
P = Variable(N, nonneg=True)
constraints = []

# SOCP constraints
for k in range(N):
    interference = sum([h[k, j] * P[j] for j in range(N) if j != k])
    constraints.append(h[k, k] * P[k] >= norm([noise_power, interference]))

constraints.append(sum(P) <= P_max)

# Objective
objective = Maximize(-sum(alpha / P))

# Solve
problem = Problem(objective, constraints)
problem.solve()

print("Optimal Power (SOCP):", P.value)

ValueError: The input must be a single CVXPY Expression, not a list. Combine Expressions using atoms such as bmat, hstack, and vstack.

In [1]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

# 参数设置
U = 7  # 用户数量
C = 7  # 基站数量
P = 50  # 最大功率 (dBm)
Max_iter = 50  # 最大迭代次数
epsilon = 1e-5  # 收敛阈值
B = 10e6  # 带宽 (Hz)

# 功率转换函数 (dBm 转线性功率)
def db2pow(db):
    return 10 ** (db / 10)

# 随机生成信道增益 (假设的信道矩阵，用户需要根据具体情况生成)
np.random.seed(1)
H_gain = np.random.rand(U, C)

# 初始化变量
p_temp = np.full(C, db2pow(P) / 2)  # 等功率初始化
A = np.ones((U, C)) - np.eye(U)  # 干扰矩阵
sum_rate = []  # 记录每次迭代的和速率
sum_rate_old = 100  # 初始和速率
iter_count = 0

# 迭代优化
while iter_count < Max_iter:
    iter_count += 1

    # Step 1: 计算中间变量 y_star
    interference = H_gain @ p_temp - np.diag(H_gain) * p_temp
    y_star = np.sqrt(np.diag(H_gain) * p_temp) / (interference + 1)

    # Step 2: 使用 cvxpy 优化功率分配
    p = cp.Variable(C, nonneg=True)
    objective = cp.sum(
        cp.log(
            1 + 2 * y_star * cp.sqrt(cp.diag(H_gain) @ p) - y_star ** 2 * (H_gain @ p + 1)
        )
    )
    constraints = [
        p >= 0,
        p <= db2pow(P),
    ]
    problem = cp.Problem(cp.Maximize(objective), constraints)
    problem.solve(solver=cp.SCS)

    # 检查求解结果
    if p.value is not None and not np.isnan(problem.value):
        sum_rate.append(problem.value)

    # 更新功率分配
    p_temp = p.value

    # 判断收敛
    if abs(problem.value - sum_rate_old) / sum_rate_old < epsilon:
        break
    else:
        sum_rate_old = problem.value

# 绘图
plt.plot(range(1, len(sum_rate) + 1), np.array(sum_rate) / np.log(2) * B / 1e6, '-b*', linewidth=1)
plt.grid(True)
plt.xlabel('Iteration number')
plt.ylabel('Sum rate (Mbps)')
plt.title('Sum Rate Convergence')
plt.show()


d:\Program\Anaconda\envs\spin\lib\site-packages\cvxpy\expressions\expression.py:650: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)
d:\Program\Anaconda\envs\spin\lib\site-packages\cvxpy\problems\problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
d:\Program\Anaconda\envs\spin\lib\site-packages\cvxpy\expressions\expression.py:650: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and 

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [12]:
H_gain = np.random.rand(7,7)
p_temp = np.full(C, db2pow(P) / 2)
c=np.diag(H_gain) * p_temp
print((H_gain @ p_temp - np.diag(H_gain) * p_temp))
print((H_gain @ p_temp - np.diag(H_gain) * p_temp)**2)
cp.sqrt(cp.diag(H_gain) @ p)

[144940.24465912 101823.69706142 203884.31801722 121106.8270244
 209247.21876251 126129.63523296 121570.01248282]
[2.10076745e+10 1.03680653e+10 4.15688151e+10 1.46668636e+10
 4.37843986e+10 1.59086849e+10 1.47792679e+10]


Expression(CONCAVE, NONNEGATIVE, ())

In [14]:
import cvxpy as cp
import numpy as np

# 示例矩阵
A = cp.Parameter((3, 3), value=np.array([[1, 2, 3],
                                         [4, 5, 6],
                                         [7, 8, 9]]))

# 提取对角线元素
diag_A = cp.diag(A)
print(diag_A)  # 输出为 [1, 5, 9]


diag_mat(param312, 0)


In [13]:
import cvxpy as cp
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

c1 = np.array([1, 1, 3, 4, 2])
c2 = np.array([-8, -2, -3, -1, -2])
a = np.array([[1,1,1,1,1], [1,2,2,1,6], [2,1,6,0,0], [0,0,1,1,5]])

b = np.array([400, 800, 200, 200])

x = cp.Variable(5, integer=True)

obj = cp.Minimize(c1*x**2+c2*x)

con = [0 <= x, x <= 99, a@x <= b]

prob = cp.Problem(obj, con)

prob.solve(solver='ECOS_BB')

print(prob.value)
print(x.value)

-16.999999999538367
[4.00000052e+00 9.99995441e-01 4.78922535e-12 4.02352057e-12
 1.00000000e+00]


d:\Program\Anaconda\envs\spin\lib\site-packages\cvxpy\expressions\expression.py:650: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 3 times so far.

  warnings.warn(msg, UserWarning)
d:\Program\Anaconda\envs\spin\lib\site-packages\cvxpy\expressions\expression.py:650: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 4 times so far.

  warnings.warn(msg, UserWarning)


In [15]:
import cvxpy as cp

# 定义变量
x = cp.Variable()

# 定义目标函数
objective = cp.Minimize(x**2)

# 定义约束
constraints = [x >= 1]

# 定义优化问题
problem = cp.Problem(objective, constraints)

# 求解问题，使用 SCS 求解器
problem.solve(solver=cp.SCS)

# 输出结果
print("Optimal value:", problem.value)
print("Optimal solution:", x.value)


Optimal value: 0.9999999999272098
Optimal solution: 0.9999999999636049


In [16]:
import cvxpy as cp

# 假设y_star, H_gain, p 是已定义的 cvxpy 变量或常量
y_star = cp.Variable((C, 1))  # 例如：假设 y_star 是一个 C x 1 的变量
H_gain = cp.Variable((U, C))  # 例如：假设 H_gain 是一个 U x C 的矩阵
p = cp.Variable((C, 1))  # 例如：假设 p 是一个 C x 1 的变量

# 构建表达式
expression = cp.log(
    1 + 2 * y_star * cp.sqrt(cp.diag(H_gain) @ p) - (y_star ** 2) * (H_gain @ p + 1)
)

# 打印表达式
print(expression)


d:\Program\Anaconda\envs\spin\lib\site-packages\cvxpy\expressions\expression.py:650: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 5 times so far.

  warnings.warn(msg, UserWarning)
d:\Program\Anaconda\envs\spin\lib\site-packages\cvxpy\expressions\expression.py:650: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 6 times so far.

  warnings.warn(msg, UserWarning)


ValueError: Incompatible dimensions (7, 1) (7, 1)

In [22]:
import cvxpy as cp

x = cp.Variable()
y = cp.Variable()

objective = cp.Minimize(2*x + 3*y)
constraint = [x + y >= 10, x >= 0, y >= 0]

problem = cp.Problem(objective, constraint)
problem.solve(solver=cp.ECOS)

print("Optimal value:", problem.value)
print("Optimal variable x:", x.value)
print("Optimal variable y:", y.value)



Optimal value: 19.99999999825794
Optimal variable x: 9.999999984636883
Optimal variable y: 9.661391355327779e-09


d:\Program\Anaconda\envs\spin\lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:356: FutureWarning: 
    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add ECOS as an explicit install dependency to your project
    or switch to our new default solver, Clarabel, by either not specifying a
    solver argument or specifying ``solver=cp.CLARABEL``. To suppress this
    warning while continuing to use ECOS, you can filter this warning using
    Python's ``warnings`` module until you are using 1.6.0.
    
  warnings.warn(ECOS_DEP_DEPRECATION_MSG, FutureWarning)


In [26]:
def optimal_power(n, a_val, b_val, P_tot=1.0, W_tot=1.0):
    # 输入参数: α 和 β 是 R_i 方程中的常数
    n = len(a_val)
    if n != len(b_val):
        print('alpha 和 beta 向量必须具有相同的长度！')
        return 'failed', np.nan, np.nan, np.nan

    P = cp.Variable(shape=n)
    W = cp.Variable(shape=n)
    alpha = cp.Parameter(shape=n)
    beta = cp.Parameter(shape=n)
    alpha.value = np.array(a_val)
    beta.value = np.array(b_val)

    # 这个函数将被用作目标函数，因此必须是 DCP 的；
    # 即，内部的元素乘法必须发生在 kl_div 内部，
    # 而不是外部，否则求解器无法知道它是 DCP 的...
    R = cp.kl_div(cp.multiply(alpha, W),
                  cp.multiply(alpha, W + cp.multiply(beta, P))) - \
                  cp.multiply(alpha, cp.multiply(beta, P))

    objective = cp.Minimize(cp.sum(R))
    constraints = [P>=0.0,
                   W>=0.0,
                   cp.sum(P)-P_tot==0.0,
                   cp.sum(W)-W_tot==0.0]

    prob = cp.Problem(objective, constraints)
    prob.solve()

    return prob.status, -prob.value, P.value, W.value

In [27]:
np.set_printoptions(precision=3)
n = 5               # 系统中接收器的数量

a_val = np.arange(10,n+10)/(1.0*n)  # α
b_val = np.arange(10,n+10)/(1.0*n)  # β
P_tot = 0.5
W_tot = 1.0
status, utility, power, bandwidth = optimal_power(n, a_val, b_val, P_tot, W_tot)

print('状态: {}'.format(status))
print('最优效用值 = {:.4g}'.format(utility))
print('最优功率水平:\n{}'.format(power))
print('最优带宽:\n{}'.format(bandwidth))

状态: optimal
最优效用值 = 2.451
最优功率水平:
[1.416e-09 2.139e-09 3.519e-09 8.464e-09 5.000e-01]
最优带宽:
[3.665e-09 4.761e-09 7.223e-09 1.672e-08 1.000e+00]


d:\Program\Anaconda\envs\spin\lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:237: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Disciplined Parametrized Programming, at https://www.cvxpy.org/tutorial/dpp/index.html
  warnings.warn(DPP_ERROR_MSG)
